In [1]:
import pandas as pd
import numpy as np

train_data = pd.read_csv('train.csv')
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
train_data = train_data[['PassengerId', 'Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]

In [3]:
train_data = train_data.dropna(axis = 0, subset = ['Age'] )
#train_data = train_data.fillna(value={'Age': 0})

In [4]:
train_data.isnull().any()

PassengerId    False
Survived       False
Pclass         False
Sex            False
Age            False
SibSp          False
Parch          False
Fare           False
Embarked        True
dtype: bool

In [5]:
train_data.loc[train_data.Sex == 'male', 'Sex'] = 1
train_data.loc[train_data.Sex == 'female', 'Sex'] = 0

In [6]:
train_data.loc[train_data.Embarked == 'S', 'Embarked'] = 1
train_data.loc[train_data.Embarked == 'C', 'Embarked'] = 2
train_data.loc[train_data.Embarked == 'Q', 'Embarked'] = 3

In [7]:
train_data = train_data.fillna(value={'Embarked': 0})
#train_data = train_data.dropna(axis = 0, subset = ['Embarked'] )

In [8]:
train_data.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,1,22.0,1,0,7.2500,1
1,2,1,1,0,38.0,1,0,71.2833,2
2,3,1,3,0,26.0,0,0,7.9250,1
3,4,1,1,0,35.0,1,0,53.1000,1
4,5,0,3,1,35.0,0,0,8.0500,1


In [9]:
train_data.isnull().any()

PassengerId    False
Survived       False
Pclass         False
Sex            False
Age            False
SibSp          False
Parch          False
Fare           False
Embarked       False
dtype: bool

In [10]:
id_set = train_data['PassengerId']
label_set = train_data['Survived']
data_set = train_data[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]

In [11]:
data_set.isnull().any()

Pclass      False
Sex         False
Age         False
SibSp       False
Parch       False
Fare        False
Embarked    False
dtype: bool

In [12]:
from sklearn.model_selection import train_test_split
x = data_set
y = label_set
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=0)

In [13]:
print(x_train.shape)
print(type(y_train))
print(x_test.shape)
print(y_test.shape)

(499, 7)
<class 'pandas.core.series.Series'>
(215, 7)
(215,)


In [14]:
y_train = y_train.as_matrix().reshape((499, 1))
y_test = y_test.as_matrix().reshape((215, 1))

C:\program\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
C:\program\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [15]:
def get_batch(data_x,data_y,batch_size=32):
    batch_n=len(data_x)//batch_size
    for i in range(batch_n):
        batch_x=data_x[i*batch_size:(i+1)*batch_size]
        batch_y=data_y[i*batch_size:(i+1)*batch_size]
        
        yield batch_x,batch_y

In [16]:
import tensorflow as tf

learning_rate = 0.1
num_steps = 500
batch_size = 100
display_step = 100

n_hidden_1 = 10
num_input = 7
num_classes = 1

x = tf.placeholder(tf.float32, [None, num_input])
y = tf.placeholder(tf.float32, [None, num_classes])

weights = {
    'h1': tf.Variable(tf.random_normal([num_input, n_hidden_1])),
    'out': tf.Variable(tf.random_normal([n_hidden_1, num_classes]))
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

def neural_network(x):
    layer_1 = tf.nn.relu(tf.add(tf.matmul(x, weights['h1']), biases['b1']))
    out_layer = tf.add(tf.matmul(layer_1, weights['out']), biases['out'])
    return out_layer

logits = neural_network(x)

# loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=y))
# loss_op = tf.reduce_mean((logits - y) ** 2)
cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=logits)
loss_op = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

predicted = tf.nn.sigmoid(logits)
correct_pred = tf.equal(tf.round(predicted), y)
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

Instructions for updating:
Colocations handled automatically by placer.


In [18]:
epochs = 200
train_collect = 50
train_print=train_collect*2

learning_rate_value = 0.001
batch_size=16

x_collect = []
train_loss_collect = []
train_acc_collect = []
valid_loss_collect = []
valid_acc_collect = []

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    iteration=0
    for e in range(epochs):
        for batch_x,batch_y in get_batch(x_train,y_train,batch_size):
            iteration+=1
            _, train_loss, train_acc = sess.run([train_op, loss_op, accuracy], feed_dict={x: x_train, y: y_train})
            if iteration % train_collect == 0:
                x_collect.append(e)
                train_loss_collect.append(train_loss)
                train_acc_collect.append(train_acc)

                if iteration % train_print==0:
                     print("Epoch: {}/{}".format(e + 1, epochs),
                      "Train Loss: {:.4f}".format(train_loss),
                      "Train Acc: {:.4f}".format(train_acc))
                
                val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={x: x_test, y: y_test})
                valid_loss_collect.append(val_loss)
                valid_acc_collect.append(val_acc)
                
                if iteration % train_print==0:
                    print("Epoch: {}/{}".format(e + 1, epochs),
                      "Validation Loss: {:.4f}".format(val_loss),
                      "Validation Acc: {:.4f}".format(val_acc))
    print("Accuracy: ", sess.run(accuracy, feed_dict={x: x_test, y: y_test}))

Epoch: 4/200 Train Loss: 0.5738 Train Acc: 0.7796
Epoch: 4/200 Validation Loss: 0.5278 Validation Acc: 0.7953
Epoch: 7/200 Train Loss: 0.6198 Train Acc: 0.7756
Epoch: 7/200 Validation Loss: 0.5492 Validation Acc: 0.8000
Epoch: 10/200 Train Loss: 0.5251 Train Acc: 0.7896
Epoch: 10/200 Validation Loss: 0.4841 Validation Acc: 0.7860
Epoch: 13/200 Train Loss: 0.5031 Train Acc: 0.7856
Epoch: 13/200 Validation Loss: 0.5028 Validation Acc: 0.8000
Epoch: 17/200 Train Loss: 0.4139 Train Acc: 0.8196
Epoch: 17/200 Validation Loss: 0.4877 Validation Acc: 0.8093
Epoch: 20/200 Train Loss: 0.3975 Train Acc: 0.8297
Epoch: 20/200 Validation Loss: 0.4661 Validation Acc: 0.8000
Epoch: 23/200 Train Loss: 0.3966 Train Acc: 0.8236
Epoch: 23/200 Validation Loss: 0.4672 Validation Acc: 0.8047
Epoch: 26/200 Train Loss: 0.4134 Train Acc: 0.8156
Epoch: 26/200 Validation Loss: 0.5285 Validation Acc: 0.7953
Epoch: 30/200 Train Loss: 0.3865 Train Acc: 0.8277
Epoch: 30/200 Validation Loss: 0.4797 Validation Acc: 0.7

In [81]:
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "QDA"]

classifiers = [
    KNeighborsClassifier(8),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]

for name, clf in zip(names, classifiers):
    clf.fit(x_train, y_train)
    score = clf.score(x_test, y_test)
    print(name," ===> ",  score)

Nearest Neighbors  ===>  0.641860465116279
Linear SVM  ===>  0.8046511627906977
RBF SVM  ===>  0.5720930232558139
Gaussian Process  ===>  0.8046511627906977
Decision Tree  ===>  0.7906976744186046
Random Forest  ===>  0.7534883720930232
Neural Net  ===>  0.7674418604651163
AdaBoost  ===>  0.8186046511627907
Naive Bayes  ===>  0.7813953488372093
QDA  ===>  0.786046511627907
